In [1]:
import numpy as np
PI2 = 2 * np.pi

## Упражнение 1

В этой главе показано, как выразить ДПФ и обратное ДПФ произведениями матриц. Время выполнения этих операций пропорционально N^2, где N - длина массива сигнала. Во многих случаях этого достаточно, но есть алгоритм и побыстрее - быстрое преобразование Фурье (БПФ); время его работы пропорционально N \log N.
Ключ к БПФ - это лемма Дэниелсона-Ланцоша

DFT(y)[n] = DFT(e)[n] + \exp(-2 \pi i n / N) DFT(o)[n]

Где DFT(y)[n] это n-й элемент ДПФ от y, e и o - массивы сигнала, содержащие соответственно четные и нечетные элементы y.

Эта лемма предлагает рекурсивный алгоритм для ДПФ:
  1. Дан массив сигнала y. Разделим его на четные элементы e и нечетные элементы o.
  2. Вычислим DFT e и o, делая рекурсивные вызовы.
  3. Вычислим DFT(y) для каждого значения n, используя лемму Дэниелсона-Ланцоша.

В простейшем случае эту рекурсию надо продолжать, пока длина y не дойдет до 1. Тогда DFT(y) = y. А если длина y достаточно мала, можно вычислить его ДПФ перемножением матриц используя заранее вычисленные матрицы.

## Решение

Для теста начнем с небольшого реального сигнала и вычислим его БПФ:

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Реализация ДПФ из книги:

In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

Мы можем подтвердить, что эта реализация дает тот же результат.

In [4]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

В качестве шага к созданию рекурсивного БПФ начнем с версии, которая разбивает входной массив и использует `np.fft.fft` для вычисления БПФ половин.

In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Получаем те же результаты:

In [6]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Теперь мы можем заменить `np.fft.fft` рекурсивными вызовами и добавить базовый вариант:

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Получаем те же результаты:

In [8]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16